In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

from datasets import load_from_disk
# from peft import PeftModel
from utils import *

os.environ['CUDA_VISIBLE_DEVICES'] = "5,6"

In [2]:
base_model = AutoModelForCausalLM.from_pretrained(
    'meta-llama/Llama-2-7b-chat-hf',
    trust_remote_code=True,
    device_map="auto",
    torch_dtype=torch.float16,   
)
base_model.model_parellal = True

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.95s/it]


In [3]:
from peft import PeftModel

model = PeftModel.from_pretrained(base_model, 'LUMOS/lumos-forecaster_dow30_llama2-7b_lora')
model = model.eval()

In [4]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-chat-hf')
tokenizer.padding_side = "right"
tokenizer.pad_token_id = tokenizer.eos_token_id

In [5]:
test_dataset = load_from_disk('./data/lumos-forecaster-crypto-20230131-20231231-1-4-08/')['test']

In [6]:
def test_demo(model, tokenizer, prompt):

    inputs = tokenizer(
        prompt, return_tensors='pt',
        padding=False, max_length=4096
    )
    inputs = {key: value.to(model.device) for key, value in inputs.items()}
        
    res = model.generate(
        **inputs, max_length=4096, do_sample=True,
        eos_token_id=tokenizer.eos_token_id,
        use_cache=True
    )
    output = tokenizer.decode(res[0], skip_special_tokens=True)
    return output    
    # return res

In [ ]:
answers, gts = [], []

for i in range(len(test_dataset)):
    prompt = test_dataset[i]['prompt']
    output = test_demo(model, tokenizer, prompt)
    answer = re.sub(r'.*\[/INST\]\s*', '', output, flags=re.DOTALL)
    gt = test_dataset[i]['answer']
    print('\n------- Prompt ------\n')
    print(prompt)
    print('\n------- LLaMA2 Finetuned ------\n')
    print(answer)
    print('\n------- GPT4 Groundtruth ------\n')
    print(gt)
    print('\n===============\n')
    answers.append(answer)
    gts.append(gt)

In [9]:
gts[0]

"[Positive Developments]:\n1. American Express Co has been recognized as part of Berkshire Hathaway’s holdings, adding a level of investor faith due to Warren Buffet's renowned investment strategies.\n2. The company's CEO, Stephen J. Squeri, announced his participation in the Bernstein 39th Annual Strategic Decisions Conference, which can potentially bring new strategic insights and boost investor confidence in the company's management. \n3. The stock has experienced a recent rise, hinting at a bullish momentum that could potentially continue.\n\n[Potential Concerns]:\n1. Debt ceiling talks might potentially impact stock performance, if negotiations lead to stock market volatility.\n2. No basic financials reported in the past week, which makes it challenging to effectively assess the company's current performance.\n\n[Prediction & Analysis]:\nPrediction: Up by 2-3%\nAnalysis: Based on the positive and negative factors analyzed above, a prediction for a slight uptrend in the American Ex

In [10]:
answers[0]

"[Positive Developments]:\n1. The stock price of American Express Co. has been steadily rising in the past weeks, indicating an increasing demand for the company's stock.\n2. The company's Chairman and CEO, Stephen J. Squeri, is scheduled to participate in the Bernstein 39th Annual Strategic Decisions Conference, which could lead to increased investor interest and confidence in the company's strategy and financial performance.\n3. The company's U.S. Consumer Services segment is expected to see increased revenue due to higher loan volumes, which could indicate a strong financial performance.\n\n[Potential Concerns]:\n1. The company did not report any basic financials in the last week, which makes it difficult to assess the company's financial health.\n2. The company's stock price is still underperforming the market, indicating a lack of investor confidence.\n3. The company's stock price has been rising gradually but at a relatively slow pace, which could indicate a lack of significant u

In [11]:
calc_metrics(answers, gts)


Binary Accuracy: 0.29  |  Mean Square Error: 4.64

Rouge Score of Positive Developments: {'rouge1': 0.4202282600542612, 'rouge2': 0.14931479501035597, 'rougeL': 0.2578559230218219}

Rouge Score of Potential Concerns: {'rouge1': 0.40652271907403026, 'rouge2': 0.13952370414379708, 'rougeL': 0.2513779865861211}

Rouge Score of Summary Analysis: {'rouge1': 0.4178582895911193, 'rouge2': 0.11082203617356211, 'rougeL': 0.2031596404349372}


{'valid_count': 14,
 'bin_acc': 0.2857142857142857,
 'mse': 4.642857142857143,
 'pros_rouge_scores': {'rouge1': 0.4202282600542612,
  'rouge2': 0.14931479501035597,
  'rougeL': 0.2578559230218219},
 'cons_rouge_scores': {'rouge1': 0.40652271907403026,
  'rouge2': 0.13952370414379708,
  'rougeL': 0.2513779865861211},
 'anal_rouge_scores': {'rouge1': 0.4178582895911193,
  'rouge2': 0.11082203617356211,
  'rougeL': 0.2031596404349372}}

In [14]:
print(answers[0])

[Positive Developments]:
1. The stock price of American Express Co. has been steadily rising in the past weeks, indicating an increasing demand for the company's stock.
2. The company's Chairman and CEO, Stephen J. Squeri, is scheduled to participate in the Bernstein 39th Annual Strategic Decisions Conference, which could lead to increased investor interest and confidence in the company's strategy and financial performance.
3. The company's U.S. Consumer Services segment is expected to see increased revenue due to higher loan volumes, which could indicate a strong financial performance.

[Potential Concerns]:
1. The company did not report any basic financials in the last week, which makes it difficult to assess the company's financial health.
2. The company's stock price is still underperforming the market, indicating a lack of investor confidence.
3. The company's stock price has been rising gradually but at a relatively slow pace, which could indicate a lack of significant upward mom